In [ ]:
# default_exp preprocessing.clean

In [ ]:
#all_slow

# Clean

> Functions to split the raw EHR dataset, clean and save for further processing & vocab creation.

In [ ]:
#hide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
from lemonpie.basics import *
from lemonpie.preprocessing import clean
from fastai.imports import *
import ray

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
ray.init()

2022-09-19 13:24:21,109	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.86.91',
 'raylet_ip_address': '192.168.86.91',
 'redis_address': '192.168.86.91:6379',
 'object_store_address': '/tmp/ray/session_2022-09-19_13-24-19_530925_6659/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-09-19_13-24-19_530925_6659/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-09-19_13-24-19_530925_6659',
 'metrics_export_port': 43325,
 'node_id': '1862fc82882e08a854d0d0837b34696cdf4245fb0fd053b13c0e8208'}

In [ ]:
COHERENT_DATA_STORE = '/home/vinod/code/datasets/coherent'
COHERENT_DATAGEN_DATE = '08-10-2021'

# Coherent Preprocessing

**Retain only patients with FHIR bundles.**

In [ ]:
def retain_fhir_patients(coherent_path = COHERENT_DATA_STORE):
    """Retain only patients with FHIR bundles."""

    file_list = os.listdir(f'{coherent_path}/output/fhir')
    pids = [((file).split("_")[-1]).split(".")[0] for file in file_list]
    fhir_pids = pd.DataFrame(pids, columns=['Id'])

    old_pts = pd.read_csv(f"{COHERENT_DATA_STORE}/output/csv/patients.csv", low_memory=False)
    fhir_pts = old_pts.merge(fhir_pids, on="Id", how="inner")
    assert len(fhir_pts) == len(fhir_pids)
    fhir_pts.to_csv(f"{COHERENT_DATA_STORE}/raw_original/patients.csv")
    return


In [ ]:
retain_fhir_patients()

**Remove ECG from observations and create ecg.csv**

In [ ]:
def moveout_ecg(coherent_path = COHERENT_DATA_STORE):
    """Move ECG data out of Observations into its own csv."""
    old_obs = pd.read_csv(f"{COHERENT_DATA_STORE}/output/csv/observations.csv", low_memory=False)
    ecg_obs = old_obs[old_obs["CODE"] == "29303009"]
    new_obs = old_obs.drop(ecg_obs.index)
    assert len(new_obs) == len(old_obs) - len(ecg_obs)
    new_obs.to_csv(f"{COHERENT_DATA_STORE}/raw_original/observations.csv")
    ecg_obs.reset_index(inplace=True, drop=True)
    odd_indxs = [i for i in range(1, len(ecg_obs), 2)]
    ecg_obs.drop(odd_indxs, inplace=True)
    ecg_obs.drop(columns=["ENCOUNTER", "CODE", "DESCRIPTION", "UNITS", "TYPE"], inplace=True)
    ecg_obs.to_csv(f"{COHERENT_DATA_STORE}/ecg.csv")

In [ ]:
moveout_ecg()

/home/vinod/anaconda3/envs/lemonpie/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
file_list = os.listdir(f'{COHERENT_DATA_STORE}/output/dna')
dna_pids = [file.split("_")[2] for file in file_list]

In [ ]:
len(dna_pids)

889

In [ ]:
patients.set_index("Id", inplace=True)

In [ ]:
patients.loc[pids.index]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['Lebrón484', 'Estévez304', 'Ulloa39'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

## Split 

Once dataset is assembled, the folder will look as follows .. 

In [ ]:
os.listdir(f"{COHERENT_DATA_STORE}/raw_original")

['patients.csv',
 'observations.csv',
 'allergies.csv',
 'payers.csv',
 'careplans.csv',
 'medications.csv',
 'devices.csv',
 'organizations.csv',
 'imaging_studies.csv',
 'procedures.csv',
 'payer_transitions.csv',
 'supplies.csv',
 'conditions.csv',
 'providers.csv',
 'encounters.csv',
 'immunizations.csv']

In [ ]:
#export
def read_raw_ehrdata(path, csv_names = FILENAMES):
    '''Read raw EHR data'''
    dfs = [pd.read_csv(f'{path}/{fname}.csv', low_memory=False) for fname in csv_names]
    return dfs    

In [ ]:
dfs = read_raw_ehrdata(f"{COHERENT_DATA_STORE}/raw_original")

In [ ]:
patients, observations, allergies, careplans, medications, imaging_studies, procedures, conditions, immunizations = dfs

In [ ]:
patients

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,1957-04-07,NaN,999-32-2242,S99944366,X13210523X,Mr.,Gregg522,Cummerata161,NaN,...,Leominster Massachusetts US,648 Gutkowski Parade,Amherst,Massachusetts,Hampshire County,NaN,42.392261,-72.479152,1318721.22,4100.44
1,b8eb8d31-1031-fb5b-e207-b9815f80744c,1975-08-16,NaN,999-70-2742,S99952609,X70704838X,Mr.,Lemuel304,Rodriguez71,NaN,...,Osaka Osaka JP,496 Deckow Mill Suite 32,Framingham,Massachusetts,Middlesex County,1702.0,42.263233,-71.481261,9089.75,0.00
2,ce9bd436-6b59-0452-86a4-61f3642736bc,1945-05-11,2015-04-09,999-54-1330,S99979547,X86639992X,Mr.,Leonardo412,Klein929,NaN,...,Marlborough Massachusetts US,314 Brakus Forge Unit 94,New Bedford,Massachusetts,Bristol County,2744.0,41.665030,-70.914507,230547.34,5821.16
3,6fc3e360-ae68-c411-e091-4734df51eb18,1947-12-30,NaN,999-59-9652,S99949959,X34069329X,Mrs.,Adelle43,Dooley940,NaN,...,North Amherst Massachusetts US,779 Haag Loaf,Wellesley,Massachusetts,Norfolk County,2457.0,42.311830,-71.286312,1568950.96,8597.39
4,ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,1993-02-05,NaN,999-11-2438,S99932651,X80439009X,Mrs.,Veta780,Spencer878,NaN,...,Lowell Massachusetts US,222 Fisher Dam Apt 74,New Bedford,Massachusetts,Bristol County,2747.0,41.671645,-70.914871,615324.99,9184.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3534,11d3a003-57bf-d281-60b0-8ba6a523c557,1931-08-02,2007-01-07,999-91-5967,S99981244,X25899378X,Mr.,Bernardo699,Quesada500,NaN,...,Ponce Puerto Rico PR,879 Mayer Underpass,Saugus,Massachusetts,Essex County,1906.0,42.481352,-71.032619,44976.63,3564.72
3535,d70cac07-9852-8260-1daa-6dc227f70b39,1931-08-02,1994-12-09,999-31-9876,S99952564,X34227013X,Mr.,Gilberto712,Jasso472,NaN,...,Port-au-Prince Haiti HT,592 Pfeffer Avenue Unit 66,Saugus,Massachusetts,Essex County,1906.0,42.484434,-71.044432,58376.03,0.00
3536,09f7af75-930c-f7c1-e66b-6ff7f674865a,1931-08-02,1952-07-24,999-80-3233,S99950232,X23808238X,Mr.,Timoteo39,Fajardo595,NaN,...,Portsmouth Saint John Parish DM,526 Lemke Dale Apt 15,Saugus,Massachusetts,Essex County,NaN,42.485559,-71.040059,4576.98,0.00
3537,4c84197b-e4e9-b971-e642-52ee9bb102cf,1931-08-02,NaN,999-47-5802,S99910764,X73803844X,Mr.,Joaquín233,Irizarry542,NaN,...,Caguas Puerto Rico PR,768 Emmerich Walk,Saugus,Massachusetts,Essex County,1906.0,42.424218,-70.982313,96057.19,3641.53


In [ ]:
#export
def split_patients(patients, valid_pct=0.2, test_pct=0.2, random_state=1234):
    '''Split the patients dataframe'''
    train_pct = 1 - (valid_pct + test_pct)
    print(f'Splits:: train: {train_pct}, valid: {valid_pct}, test: {test_pct}')
    patients = patients.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return np.split(patients, [int(train_pct*len(patients)), int((train_pct+valid_pct)*len(patients))])

In [ ]:
train_pts, valid_pts, test_pts = split_patients(patients, .2,.1)

Splits:: train: 0.7, valid: 0.2, test: 0.1


In [ ]:
len(patients), len(train_pts), len(valid_pts), len(test_pts)

(3539, 2477, 708, 354)

In [ ]:
assert len(patients) == len(train_pts)+len(valid_pts)+len(test_pts)

In [ ]:
#export
def split_ehr_dataset(path, valid_pct=0.2, test_pct=0.2, random_state=1234):
    '''Split EHR dataset into train, valid, test and save'''

    train_dfs, valid_dfs, test_dfs = [],[],[]
    
    dfs = read_raw_ehrdata(f'{path}/raw_original')
    all_pts = dfs[0]
    all_pts.rename(str.lower, axis='columns', inplace=True)
    train_pt, valid_pt, test_pt = split_patients(dfs[0], valid_pct, test_pct, random_state)
    train_dfs.append(train_pt)
    valid_dfs.append(valid_pt)
    test_dfs.append(test_pt)
    print(f'Split {FILENAMES[0]} into:: Train: {len(train_pt)}, Valid: {len(valid_pt)}, Test: {len(test_pt)} -- Total before split: {len(dfs[0])}')
    
    for df, name in zip(dfs[1:], FILENAMES[1:]):
        df = df.set_index('PATIENT')
        df_train = df.loc[df.index.intersection(train_pt['id']).unique()]
        df_valid = df.loc[df.index.intersection(valid_pt['id']).unique()]
        df_test = df.loc[df.index.intersection(test_pt['id']).unique()]
        assert len(df) == len(df_train)+len(df_valid)+len(df_test),f'Split failed {name}: {len(df)} != {len(df_train)}+{len(df_valid)}+{len(df_test)}'
        train_dfs.append(df_train.reset_index())
        valid_dfs.append(df_valid.reset_index())
        test_dfs.append(df_test.reset_index())

    
    for split in ['train', 'valid', 'test']:
        d = Path(f'{path}/raw_split/{split}')
        d.mkdir(parents=True, exist_ok=True)
        
        if split == 'train':
            for df, name in zip(train_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved train data to {d}')
        
        if split == 'valid':
            for df, name in zip(valid_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved valid data to {d}')
    
        if split == 'test':
            for df, name in zip(test_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved test data to {d}')

In [ ]:
split_ehr_dataset(COHERENT_DATA_STORE) #will use default values for split percents

Splits:: train: 0.6, valid: 0.2, test: 0.2
Split patients into:: Train: 2123, Valid: 708, Test: 708 -- Total before split: 3539
Saved train data to /home/vinod/code/datasets/coherent/raw_split/train
Saved valid data to /home/vinod/code/datasets/coherent/raw_split/valid
Saved test data to /home/vinod/code/datasets/coherent/raw_split/test


In [ ]:
def load_split_data(path):
    all_dfs = []
    for split in ['train', 'valid', 'test']:
            split_path = f'{path}/raw_split/{split}'
            dfs = read_raw_ehrdata(split_path) 
            all_dfs.append(dfs)
    return all_dfs

In [ ]:
train_dfs, valid_dfs, test_dfs = load_split_data(COHERENT_DATA_STORE)

## Clean

In [ ]:
#export
@ray.remote(num_returns=3)
def cleanup_pts(pts, is_train, today=None):
    '''Clean patients df'''
    
    pts.rename(str.lower, axis='columns', inplace=True)
    pts = pts.loc[:, ['id', 'birthdate', 'marital', 'race', 'ethnicity', 'gender', 'birthplace', 'city', 'state', 'zip']]
    pts.rename(columns={"id":"patient"}, inplace=True)
    pts = pts.astype({'birthdate':'datetime64'}) 
    pts['zip'] = pts['zip'].fillna(0.0).astype(int)    
    if today == None: today = pd.Timestamp.today()
    else            : today = pd.to_datetime(today)
    pts['age_now_days'] = pts['birthdate'].apply(lambda bday: (today-bday).days)
    
    pts.fillna('xxxnan', inplace=True)
    if is_train: pt_codes = pts.drop(columns=['patient'], inplace=False)
    pts.set_index('patient', inplace=True)
    pt_demographics = pts
    patients = pts.loc[:,['birthdate']]
    
    return [patients, pt_demographics, pt_codes] if is_train else [patients, pt_demographics, None]

`patients` data frame looks like this before cleanup.. 

In [ ]:
patients.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,1957-04-07,NaN,999-32-2242,S99944366,X13210523X,Mr.,Gregg522,Cummerata161,NaN,...,Leominster Massachusetts US,648 Gutkowski Parade,Amherst,Massachusetts,Hampshire County,NaN,42.392261,-72.479152,1318721.22,4100.44
1,b8eb8d31-1031-fb5b-e207-b9815f80744c,1975-08-16,NaN,999-70-2742,S99952609,X70704838X,Mr.,Lemuel304,Rodriguez71,NaN,...,Osaka Osaka JP,496 Deckow Mill Suite 32,Framingham,Massachusetts,Middlesex County,1702.0,42.263233,-71.481261,9089.75,0.00
2,ce9bd436-6b59-0452-86a4-61f3642736bc,1945-05-11,2015-04-09,999-54-1330,S99979547,X86639992X,Mr.,Leonardo412,Klein929,NaN,...,Marlborough Massachusetts US,314 Brakus Forge Unit 94,New Bedford,Massachusetts,Bristol County,2744.0,41.665030,-70.914507,230547.34,5821.16
3,6fc3e360-ae68-c411-e091-4734df51eb18,1947-12-30,NaN,999-59-9652,S99949959,X34069329X,Mrs.,Adelle43,Dooley940,NaN,...,North Amherst Massachusetts US,779 Haag Loaf,Wellesley,Massachusetts,Norfolk County,2457.0,42.311830,-71.286312,1568950.96,8597.39
4,ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,1993-02-05,NaN,999-11-2438,S99932651,X80439009X,Mrs.,Veta780,Spencer878,NaN,...,Lowell Massachusetts US,222 Fisher Dam Apt 74,New Bedford,Massachusetts,Bristol County,2747.0,41.671645,-70.914871,615324.99,9184.78


In [ ]:
train_pts_cleaned = cleanup_pts.remote(train_dfs[0], is_train=True, today=COHERENT_DATAGEN_DATE)
train_pts_cleaned = ray.get(train_pts_cleaned) #train_pts_data[0], train_pts_data[1], train_pts_data[2]

In [ ]:
valid_pts_cleaned = ray.get(cleanup_pts.remote(valid_dfs[0], is_train=False, today=COHERENT_DATAGEN_DATE))

The cleanup function produces the following 3 dfs - `patients`, `pt_demographics`, `pt_codes` **for training data**

In [ ]:
for df in train_pts_cleaned:
    display(df.head(3))

,birthdate
patient,
06ba8d4c-199e-88e5-e5e9-f1d8e2e0a7c4,1928-12-12
f3b27f44-26ff-76f2-a1b2-a588a2c77474,1972-12-13
dbf1da00-04b4-b158-4900-c24b872fb362,1989-03-02


,birthdate,marital,race,ethnicity,gender,birthplace,city,state,zip,age_now_days
patient,,,,,,,,,,
06ba8d4c-199e-88e5-e5e9-f1d8e2e0a7c4,1928-12-12,M,white,nonhispanic,F,Pittsfield Massachusetts US,Acushnet Center,Massachusetts,0,33844
f3b27f44-26ff-76f2-a1b2-a588a2c77474,1972-12-13,M,white,nonhispanic,F,Uxbridge Massachusetts US,Boston,Massachusetts,2210,17772
dbf1da00-04b4-b158-4900-c24b872fb362,1989-03-02,M,white,hispanic,M,Boston Massachusetts US,Springfield,Massachusetts,1129,11849


,birthdate,marital,race,ethnicity,gender,birthplace,city,state,zip,age_now_days
0,1928-12-12,M,white,nonhispanic,F,Pittsfield Massachusetts US,Acushnet Center,Massachusetts,0,33844
1,1972-12-13,M,white,nonhispanic,F,Uxbridge Massachusetts US,Boston,Massachusetts,2210,17772
2,1989-03-02,M,white,hispanic,M,Boston Massachusetts US,Springfield,Massachusetts,1129,11849


And the cleanup function produces the following 2 dfs - `patients`, `pt_demographics` **for validation and test data**, i.e. no `pt_codes` which only come from training

In [ ]:
for df in valid_pts_cleaned[:2]:
    display(df.head(3))

,birthdate
patient,
eb4c205d-7d8a-47a2-801a-fc324634ec61,1954-12-06
15a4dfa4-a611-dd50-8dcd-b20725b0e3e9,1918-01-07
632ef37a-8d01-e00a-6cef-02d2885dfb80,1996-02-26


,birthdate,marital,race,ethnicity,gender,birthplace,city,state,zip,age_now_days
patient,,,,,,,,,,
eb4c205d-7d8a-47a2-801a-fc324634ec61,1954-12-06,M,white,nonhispanic,M,Dudley Massachusetts US,Boston,Massachusetts,2116,24354
15a4dfa4-a611-dd50-8dcd-b20725b0e3e9,1918-01-07,M,black,nonhispanic,M,Norton Massachusetts US,Boston,Massachusetts,2127,37836
632ef37a-8d01-e00a-6cef-02d2885dfb80,1996-02-26,xxxnan,white,nonhispanic,M,Wellesley Massachusetts US,Marshfield,Massachusetts,2050,9297


**The case for keeping a record of the data generation date**

Also note the difference in `age_now` if it were calculated based on default (`pd.Timestamp.today()`) vs `COHERENT_DATAGEN_DATE` which is the data generation date for this 1K dataset.  

In [ ]:
# age_now -- today vs run_date
(pd.to_datetime(pd.Timestamp.today()) - train_pts_cleaned[0].iloc[2])[0].days, (pd.to_datetime(COHERENT_DATAGEN_DATE) - train_pts_cleaned[0].iloc[2])[0].days

(12248, 11849)

In [ ]:
COHERENT_DATAGEN_DATE, pd.Timestamp.today()

('08-10-2021', Timestamp('2022-09-13 17:17:54.147529'))

In [ ]:
#export
@ray.remote(num_returns=2)
def cleanup_obs(obs, is_train):
    '''Clean observations df'''
    
    obs.rename(str.lower, axis='columns', inplace=True)
    obs.units.fillna('xxxnan', inplace=True)
    obs.dropna(subset=['value'], inplace=True)
    
    obs.rename(columns={"code":"orig_code", "description":"desc"}, inplace=True)
    obs['code'] = obs['orig_code'].str.cat(obs[['value', 'units', 'type']].astype(str), sep='||')

    if is_train: obs_codes = obs.loc[:, ['orig_code', 'desc', 'value', 'units', 'type']]
    
    obs = obs.loc[:, ['patient', 'date', 'code']]
    obs = obs.astype({'date':'datetime64'})
    obs.set_index('patient', inplace=True)
    
    return [obs, obs_codes] if is_train else [obs, None]

- Drops rows with null in the `VALUE` column
- Creates a new `code` column with a concatenation of `code`, `value`, `units` and `type`
    - so that we can use the following logic during vocab creation for observations (further detailed in the vocab documentation)

**For `numeric`**
```
for 'numeric'
    get unique 'codes'
    for each unique code
        get unique 'units'
            for each unique unit
                bucketize 'values'
                create vocab entry for each 'bucket' -- code||value_bucket||units
```
**For `text`**
```
for 'text'
    get unique 'codes'
    for each unique code
        get unique 'units' #this will be null
            for each unique unit
                get unique 'values'
                create vocab entry for each -- code||value||units
```

'observations' df before cleanup ..

In [ ]:
observations.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2012-05-06T08:57:56Z,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,3cb4d689-187e-3231-df3c-05b7f1b77bb5,8302-2,Body Height,176.7,cm,numeric
1,2005-07-08T20:04:40Z,ce9bd436-6b59-0452-86a4-61f3642736bc,8c5bec95-c1b9-2e00-ed20-a259baf8da47,8302-2,Body Height,172.2,cm,numeric
2,2012-01-21T13:11:33Z,b8eb8d31-1031-fb5b-e207-b9815f80744c,f2ab1cb7-f75e-c51b-63ba-2cc8c3cd2dd7,8302-2,Body Height,177.9,cm,numeric
3,2012-01-21T13:11:33Z,b8eb8d31-1031-fb5b-e207-b9815f80744c,f2ab1cb7-f75e-c51b-63ba-2cc8c3cd2dd7,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,0.0,{score},numeric
4,2012-05-06T08:57:56Z,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,3cb4d689-187e-3231-df3c-05b7f1b77bb5,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,3.0,{score},numeric


In [ ]:
train_obs_cleaned = ray.get(cleanup_obs.remote(train_dfs[1], is_train=True))

after cleanup..

In [ ]:
for df in train_obs_cleaned:
    display(df.head())

,date,code
patient,,
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2012-05-06 08:57:56,8302-2||176.7||cm||numeric
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2012-05-06 08:57:56,72514-3||3.0||{score}||numeric
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2012-05-06 08:57:56,29463-7||86.5||kg||numeric
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2012-05-06 08:57:56,39156-5||27.7||kg/m2||numeric
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2012-05-06 08:57:56,8462-4||81.0||mm[Hg]||numeric


,orig_code,desc,value,units,type
0,8302-2,Body Height,176.7,cm,numeric
1,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,3.0,{score},numeric
2,29463-7,Body Weight,86.5,kg,numeric
3,39156-5,Body Mass Index,27.7,kg/m2,numeric
4,8462-4,Diastolic Blood Pressure,81.0,mm[Hg],numeric


In [ ]:
test_obs_cleaned = ray.get(cleanup_obs.remote(test_dfs[1], is_train=False))

after cleanup..

In [ ]:
for df in test_obs_cleaned[:1]:
    display(df.head())

,date,code
patient,,
6fc3e360-ae68-c411-e091-4734df51eb18,2012-05-02 02:15:19,8302-2||166.9||cm||numeric
6fc3e360-ae68-c411-e091-4734df51eb18,2012-05-02 02:15:19,72514-3||4.0||{score}||numeric
6fc3e360-ae68-c411-e091-4734df51eb18,2012-05-02 02:15:19,29463-7||77.0||kg||numeric
6fc3e360-ae68-c411-e091-4734df51eb18,2012-05-02 02:15:19,39156-5||27.6||kg/m2||numeric
6fc3e360-ae68-c411-e091-4734df51eb18,2012-05-02 02:15:19,8462-4||53.0||mm[Hg]||numeric


In [ ]:
#export
@ray.remote(num_returns=2)
def cleanup_algs(allergies, is_train):
    '''Clean allergies df'''
    
    allergies.rename(str.lower, axis='columns', inplace=True)
    allergies.drop(columns=['encounter'], inplace=True)
    
    stops = pd.DataFrame(allergies.loc[allergies['stop'].notnull(),:])
    allergies['code'] = allergies['code'].apply(lambda x: f'{str(x)}||START')
    stops['code'] = stops['code'].apply(lambda x: f'{str(x)}||STOP')
    allergies.drop(columns=['stop'], inplace=True)
    stops.drop(columns=['start'], inplace=True)
    allergies.rename(columns={"start":"date", "description":"desc"}, inplace=True)
    stops.rename(columns={"stop":"date", "description":"desc"}, inplace=True)
    allergies = allergies.append(stops, ignore_index=True)
    
    if is_train: alg_codes = allergies.loc[:, ['code', 'desc']]
        
    allergies.drop(columns=['desc'], inplace=True)
    allergies = allergies.astype({'date':'datetime64'})
    allergies.set_index('patient', inplace=True)
    return [allergies, alg_codes] if is_train else [allergies, None]

`allergies` have a start and stop date in the same row indicating when an allergy (indicated by its code) started and stopped (or not) for a patient. <br>
So in the cleanup, we flatten that out, meaning create new rows for stop dates. <br>
The dataframe looks as follows before cleanup..

In [ ]:
allergies.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1993-09-11,NaN,ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,7c5d7a54-12e8-f192-550c-e8ef61db7c0e,419474003,Allergy to mould
1,1993-09-11,NaN,ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,7c5d7a54-12e8-f192-550c-e8ef61db7c0e,232347008,Dander (animal) allergy
2,1993-09-11,NaN,ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,7c5d7a54-12e8-f192-550c-e8ef61db7c0e,419263009,Allergy to tree pollen
3,1993-09-11,NaN,ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,7c5d7a54-12e8-f192-550c-e8ef61db7c0e,300913006,Shellfish allergy
4,1963-06-01,NaN,69e46a15-edee-8a95-fc7b-f16e72e0aabf,8edde360-759e-601e-d8db-2a5619070965,232347008,Dander (animal) allergy


In [ ]:
train_alg_cleaned = ray.get(cleanup_algs.remote(train_dfs[2], is_train=True))

Resulting in the following output after cleanup.. 

In [ ]:
for df in train_alg_cleaned:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,1993-09-11,419474003||START
ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,1993-09-11,232347008||START
ce4ce4d8-d4e2-aca2-5a92-8ce703c5077a,1993-09-11,419263009||START


,date,code
patient,,
74e382d4-bb73-83b2-2de1-efe7c064a3e3,2019-01-02,232347008||STOP
74e382d4-bb73-83b2-2de1-efe7c064a3e3,2019-01-02,418689008||STOP
74e382d4-bb73-83b2-2de1-efe7c064a3e3,2020-02-25,91930004||STOP


,code,desc
0,419474003||START,Allergy to mould
1,232347008||START,Dander (animal) allergy
2,419263009||START,Allergy to tree pollen


,code,desc
456,232347008||STOP,Dander (animal) allergy
457,418689008||STOP,Allergy to grass pollen
458,91930004||STOP,Allergy to eggs


In [ ]:
#export
@ray.remote(num_returns=2)
def cleanup_crpls(careplans, is_train):
    '''Clean careplans df'''
    
    careplans.rename(str.lower, axis='columns', inplace=True)
    careplans = careplans.loc[:, ['start', 'stop', 'patient', 'code', 'description']]
    
    stops = pd.DataFrame(careplans.loc[careplans['stop'].notnull(),:])
    careplans['code'] = careplans['code'].apply(lambda x: f'{str(x)}||START')
    stops['code'] = stops['code'].apply(lambda x: f'{str(x)}||STOP')
    careplans.drop(columns=['stop'], inplace=True)
    stops.drop(columns=['start'], inplace=True)
    careplans.rename(columns={"start":"date", "description":"desc"}, inplace=True)
    stops.rename(columns={"stop":"date", "description":"desc"}, inplace=True)
    careplans = careplans.append(stops, ignore_index=True)
    
    if is_train: crpl_codes = careplans.loc[:, ['code', 'desc']]

    careplans.drop(columns=['desc'], inplace=True)
    careplans = careplans.astype({'date':'datetime64'})
    careplans.set_index('patient', inplace=True)
    return [careplans, crpl_codes] if is_train else [careplans, None]

In [ ]:
careplans.head()

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,a6fa2d54-bfc7-677d-43e9-8f9639eb1548,2001-04-22,NaN,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,7cfc42ac-357a-c88e-1275-9d63f900b7d8,698360004,Diabetes self management plan,15777000.0,Prediabetes
1,cf59dfa7-6c8e-2bbf-9cf9-37e9f87cc4a8,2011-09-01,2011-09-15,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,92ad6010-058a-0362-d0df-86fcad3edf80,91251008,Physical therapy procedure,70704007.0,Sprain of wrist
2,6b83bab0-fcdc-c5cd-3579-a6bd7d6e261b,1993-10-09,NaN,b8eb8d31-1031-fb5b-e207-b9815f80744c,42dbe56f-5b30-38b3-5634-a9b1a0206e44,443402002,Lifestyle education regarding hypertension,59621000.0,Hypertension
3,e7787619-cb0d-8483-5efa-9d7f89e866e4,1987-05-22,NaN,ce9bd436-6b59-0452-86a4-61f3642736bc,5145eae4-d489-09a3-c60c-a59f2a662b24,698360004,Diabetes self management plan,15777000.0,Prediabetes
4,da02b168-5220-1752-823c-7e829345be69,2006-07-14,NaN,ce9bd436-6b59-0452-86a4-61f3642736bc,a08049fb-cfe9-bbbc-9ab6-b0a2f9125a2f,386257007,Demential management,230265002.0,Familial Alzheimer's disease of early onset (d...


In [ ]:
train_crpl_cleaned = ray.get(cleanup_crpls.remote(careplans, is_train=True))

In [ ]:
for df in train_crpl_cleaned:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2001-04-22,698360004||START
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2011-09-01,91251008||START
b8eb8d31-1031-fb5b-e207-b9815f80744c,1993-10-09,443402002||START


,date,code
patient,,
d70cac07-9852-8260-1daa-6dc227f70b39,1991-08-12,736353004||STOP
d70cac07-9852-8260-1daa-6dc227f70b39,1994-01-31,736353004||STOP
09f7af75-930c-f7c1-e66b-6ff7f674865a,1948-01-17,225358003||STOP


,code,desc
0,698360004||START,Diabetes self management plan
1,91251008||START,Physical therapy procedure
2,443402002||START,Lifestyle education regarding hypertension


,code,desc
20843,736353004||STOP,Inpatient care plan (record artifact)
20844,736353004||STOP,Inpatient care plan (record artifact)
20845,225358003||STOP,Wound care


In [ ]:
#export
@ray.remote(num_returns=2)
def cleanup_meds(medications, is_train):
    '''Clean `medications` df'''
    
    medications.rename(str.lower, axis='columns', inplace=True)
    medications = medications.loc[:, ['start', 'stop', 'patient', 'code', 'description']]
    
    stops = pd.DataFrame(medications.loc[medications['stop'].notnull(),:])
    medications['code'] = medications['code'].apply(lambda x: f'{str(x)}||START')
    stops['code'] = stops['code'].apply(lambda x: f'{str(x)}||STOP')
    medications.drop(columns=['stop'], inplace=True)
    stops.drop(columns=['start'], inplace=True)
    medications.rename(columns={"start":"date", "description":"desc"}, inplace=True)
    stops.rename(columns={"stop":"date", "description":"desc"}, inplace=True)
    medications = medications.append(stops, ignore_index=True)
    
    if is_train: med_codes = medications.loc[:, ['code', 'desc']]

    medications.drop(columns=['desc'], inplace=True)
    medications = medications.astype({'date':'datetime64'})
    medications.set_index('patient', inplace=True)
    return [medications, med_codes] if is_train else [medications, None]

In [ ]:
medications.head()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2003-04-27T10:01:56Z,NaN,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,d47b3510-2895-3b70-9897-342d681c769d,155aa73b-46da-5808-c218-80a5ed671009,2001499,Vitamin B 12 5 MG/ML Injectable Solution,2.29,0.0,222,508.38,NaN,NaN
1,2011-09-01T09:16:56Z,2011-09-15T09:16:56Z,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,d47b3510-2895-3b70-9897-342d681c769d,92ad6010-058a-0362-d0df-86fcad3edf80,310965,Ibuprofen 200 MG Oral Tablet,15.55,0.0,1,15.55,NaN,NaN
2,1993-10-09T13:11:33Z,1994-10-15T13:11:33Z,b8eb8d31-1031-fb5b-e207-b9815f80744c,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,42dbe56f-5b30-38b3-5634-a9b1a0206e44,314076,lisinopril 10 MG Oral Tablet,0.02,0.0,371,7.42,59621000.0,Hypertension
3,1994-10-15T13:11:33Z,1995-10-21T13:11:33Z,b8eb8d31-1031-fb5b-e207-b9815f80744c,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,92c4fdef-9d29-5bab-6436-418daf80d4af,314076,lisinopril 10 MG Oral Tablet,0.02,0.0,371,7.42,59621000.0,Hypertension
4,1995-10-21T13:11:33Z,1996-10-26T13:11:33Z,b8eb8d31-1031-fb5b-e207-b9815f80744c,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,03730cf4-2bce-f26b-1407-31b88acaec6a,314076,lisinopril 10 MG Oral Tablet,0.02,0.0,371,7.42,59621000.0,Hypertension


In [ ]:
train_med_cleaned = ray.get(cleanup_meds.remote(medications, is_train=True))

In [ ]:
for df in train_med_cleaned:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2003-04-27 10:01:56,2001499||START
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2011-09-01 09:16:56,310965||START
b8eb8d31-1031-fb5b-e207-b9815f80744c,1993-10-09 13:11:33,314076||START


,date,code
patient,,
3107760e-1dce-a177-4122-725241aae61a,2021-07-11 13:39:54,205923||STOP
3107760e-1dce-a177-4122-725241aae61a,2021-07-14 16:39:54,205923||STOP
3107760e-1dce-a177-4122-725241aae61a,2021-07-17 19:39:54,205923||STOP


,code,desc
0,2001499||START,Vitamin B 12 5 MG/ML Injectable Solution
1,310965||START,Ibuprofen 200 MG Oral Tablet
2,314076||START,lisinopril 10 MG Oral Tablet


,code,desc
730598,205923||STOP,1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen]
730599,205923||STOP,1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen]
730600,205923||STOP,1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen]


In [ ]:
#export
@ray.remote(num_returns=2)
def cleanup_img(imaging_studies, is_train):
    '''Clean `imaging` df'''
    
    imaging_studies.rename(str.lower, axis='columns', inplace=True)
    imaging_studies.rename(columns={"bodysite_code":"code", "bodysite_description":"desc"}, inplace=True)
    if is_train: img_codes = imaging_studies.loc[:, ['code', 'desc']]
        
    imaging_studies = imaging_studies.loc[:, ['patient', 'date', 'code']]
    imaging_studies = imaging_studies.astype({'date':'datetime64'})
    imaging_studies.set_index('patient', inplace=True)
    return [imaging_studies, img_codes] if is_train else [imaging_studies, None]

In [ ]:
imaging_studies.head()

,Id,DATE,PATIENT,ENCOUNTER,BODYSITE_CODE,BODYSITE_DESCRIPTION,MODALITY_CODE,MODALITY_DESCRIPTION,SOP_CODE,SOP_DESCRIPTION
0,d87c9d2e-0ea4-3582-e440-5f58a5361ece,1934-08-10T03:07:45Z,4fdafa07-e5cb-9b2a-55c6-50902c49ccfb,e6767599-aa04-603b-14b6-4520c020c75d,40983000,Arm,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage
1,05ae52a9-2cdc-d3c6-d223-cd96dcf0c8e5,1957-05-10T18:29:58Z,9c452d24-00b0-d58f-4cd5-b82bd6695646,a1b6d1e4-70d6-0299-dd08-918f555e525c,80891009,Heart structure (body structure),US,Ultrasound,1.2.840.10008.5.1.4.1.1.3.1,Ultrasound Multiframe Image Storage
2,37088e57-9896-15c4-8096-2b972c99b902,1957-11-06T18:29:58Z,9c452d24-00b0-d58f-4cd5-b82bd6695646,c4e84f43-fd76-dc0d-c43b-913bcbce8b3e,80891009,Heart structure (body structure),US,Ultrasound,1.2.840.10008.5.1.4.1.1.3.1,Ultrasound Multiframe Image Storage
3,1d1a61db-94ef-a3d8-8f23-a028c1243ce9,1958-05-05T18:29:58Z,9c452d24-00b0-d58f-4cd5-b82bd6695646,787d29a9-2849-9fd2-175e-bda8588d9a79,51185008,Thoracic structure (body structure),CR,Computed Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage – for Presentation
4,4a4a95fb-a0d0-ebb3-e7fc-055639bff2c2,1958-11-06T18:59:58Z,9c452d24-00b0-d58f-4cd5-b82bd6695646,98e97c10-e62b-801b-78d5-85f30b182ada,80891009,Heart structure (body structure),US,Ultrasound,1.2.840.10008.5.1.4.1.1.3.1,Ultrasound Multiframe Image Storage


In [ ]:
train_img_cleaned = ray.get(cleanup_img.remote(imaging_studies, is_train=True))

In [ ]:
for df in train_img_cleaned:
    display(df.head(3))

,date,code
patient,,
4fdafa07-e5cb-9b2a-55c6-50902c49ccfb,1934-08-10 03:07:45,40983000
9c452d24-00b0-d58f-4cd5-b82bd6695646,1957-05-10 18:29:58,80891009
9c452d24-00b0-d58f-4cd5-b82bd6695646,1957-11-06 18:29:58,80891009


,code,desc
0,40983000,Arm
1,80891009,Heart structure (body structure)
2,80891009,Heart structure (body structure)


In [ ]:
#export
@ray.remote(num_returns=2)
def cleanup_procs(procedures, is_train):
    '''Clean `procedures` df'''
    
    procedures.rename(str.lower, axis='columns', inplace=True)
    procedures.rename(columns={"description":"desc"}, inplace=True)
    if is_train: proc_codes = procedures.loc[:, ['code', 'desc']]
    
    procedures = procedures.loc[:, ['patient', 'date', 'code']]
    procedures = procedures.astype({'date':'datetime64'})
    procedures.set_index('patient', inplace=True)
    return [procedures, proc_codes] if is_train else [procedures, None]

In [ ]:
procedures.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2012-04-06T09:36:56Z,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,efe28b57-e201-a19c-5008-1f178258b681,73761001,Colonoscopy,11859.06,NaN,NaN
1,2005-07-08T20:04:40Z,ce9bd436-6b59-0452-86a4-61f3642736bc,8c5bec95-c1b9-2e00-ed20-a259baf8da47,410006001,Digital examination of rectum,985.49,NaN,NaN
2,2012-01-21T13:11:33Z,b8eb8d31-1031-fb5b-e207-b9815f80744c,f2ab1cb7-f75e-c51b-63ba-2cc8c3cd2dd7,430193006,Medication Reconciliation (procedure),389.26,NaN,NaN
3,2005-07-08T20:08:40Z,ce9bd436-6b59-0452-86a4-61f3642736bc,d7ed0228-844c-b2f3-37c0-37c824066a50,65575008,Biopsy of prostate,3129.80,NaN,NaN
4,2006-07-08T20:38:40Z,ce9bd436-6b59-0452-86a4-61f3642736bc,a08049fb-cfe9-bbbc-9ab6-b0a2f9125a2f,410006001,Digital examination of rectum,1071.44,NaN,NaN


In [ ]:
train_proc_cleaned = ray.get(cleanup_procs.remote(procedures, is_train=True))

In [ ]:
for df in train_proc_cleaned:
    display(df.head(3))

,date,code
patient,,
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2012-04-06 09:36:56,73761001
ce9bd436-6b59-0452-86a4-61f3642736bc,2005-07-08 20:04:40,410006001
b8eb8d31-1031-fb5b-e207-b9815f80744c,2012-01-21 13:11:33,430193006


,code,desc
0,73761001,Colonoscopy
1,410006001,Digital examination of rectum
2,430193006,Medication Reconciliation (procedure)


In [ ]:
#export
@ray.remote(num_returns=2)
def cleanup_cnds(conditions, is_train):
    '''Clean `conditions` df'''
    
    conditions.rename(str.lower, axis='columns', inplace=True)
    conditions.drop(columns=['encounter'], inplace=True)
    stops = pd.DataFrame(conditions.loc[conditions['stop'].notnull(),:])
    conditions['code'] = conditions['code'].apply(lambda x: f'{str(x)}||START')
    stops['code'] = stops['code'].apply(lambda x: f'{str(x)}||STOP')
    conditions.drop(columns=['stop'], inplace=True)
    stops.drop(columns=['start'], inplace=True)
    conditions.rename(columns={"start":"date", "description":"desc"}, inplace=True)
    stops.rename(columns={"stop":"date","description":"desc"}, inplace=True)
    conditions = conditions.append(stops, ignore_index=True)
        
    if is_train: cnd_codes = conditions.loc[:, ['code', 'desc']]
        
    conditions.drop(columns=['desc'], inplace=True)
    conditions = conditions.astype({'date':'datetime64'})
    conditions.set_index('patient', inplace=True)
    return [conditions, cnd_codes] if is_train else [conditions, None]

In [ ]:
conditions.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1991-06-23,NaN,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,03bc904a-c655-0580-a620-264b065c01e6,162864005,Body mass index 30+ - obesity (finding)
1,2001-04-22,NaN,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,7cfc42ac-357a-c88e-1275-9d63f900b7d8,15777000,Prediabetes
2,2003-04-27,NaN,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,d3cbf195-f49a-24f5-fe79-c6dc21fb92da,271737000,Anemia (disorder)
3,1993-10-09,NaN,b8eb8d31-1031-fb5b-e207-b9815f80744c,42dbe56f-5b30-38b3-5634-a9b1a0206e44,59621000,Hypertension
4,2011-09-01,2011-09-15,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,92ad6010-058a-0362-d0df-86fcad3edf80,70704007,Sprain of wrist


In [ ]:
train_cnd_cleaned = ray.get(cleanup_cnds.remote(conditions, is_train=True))

In [ ]:
for df in train_cnd_cleaned:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,1991-06-23,162864005||START
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2001-04-22,15777000||START
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2003-04-27,271737000||START


,date,code
patient,,
4c84197b-e4e9-b971-e642-52ee9bb102cf,2012-08-03,444814009||STOP
4c84197b-e4e9-b971-e642-52ee9bb102cf,2016-10-18,195662009||STOP
3107760e-1dce-a177-4122-725241aae61a,2020-11-17,195662009||STOP


,code,desc
0,162864005||START,Body mass index 30+ - obesity (finding)
1,15777000||START,Prediabetes
2,271737000||START,Anemia (disorder)


,code,desc
49738,444814009||STOP,Viral sinusitis (disorder)
49739,195662009||STOP,Acute viral pharyngitis (disorder)
49740,195662009||STOP,Acute viral pharyngitis (disorder)


In [ ]:
#export
@ray.remote(num_returns=2)
def cleanup_immns(immunizations, is_train):
    '''Clean `immunizations` df'''
    
    immunizations.rename(str.lower, axis='columns', inplace=True)
    immunizations.rename(columns={"description":"desc"}, inplace=True)
    if is_train: imm_codes = immunizations.loc[:, ['code', 'desc']]
        
    immunizations = immunizations.loc[:, ['patient', 'date', 'code']]
    immunizations = immunizations.astype({'date':'datetime64'})
    immunizations.set_index('patient', inplace=True)
    return [immunizations, imm_codes] if is_train else [immunizations, None]

In [ ]:
immunizations.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST
0,2005-07-08T20:04:40Z,ce9bd436-6b59-0452-86a4-61f3642736bc,8c5bec95-c1b9-2e00-ed20-a259baf8da47,140,Influenza seasonal injectable preservative ...,140.52
1,2012-01-21T13:11:33Z,b8eb8d31-1031-fb5b-e207-b9815f80744c,f2ab1cb7-f75e-c51b-63ba-2cc8c3cd2dd7,140,Influenza seasonal injectable preservative ...,140.52
2,2012-05-06T08:57:56Z,8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,3cb4d689-187e-3231-df3c-05b7f1b77bb5,140,Influenza seasonal injectable preservative ...,140.52
3,2013-01-26T13:11:33Z,b8eb8d31-1031-fb5b-e207-b9815f80744c,bbe4a96d-ce87-f714-2898-ba8d001262bb,140,Influenza seasonal injectable preservative ...,140.52
4,2006-07-14T20:04:40Z,ce9bd436-6b59-0452-86a4-61f3642736bc,a08049fb-cfe9-bbbc-9ab6-b0a2f9125a2f,140,Influenza seasonal injectable preservative ...,140.52


In [ ]:
train_imm_cleaned = ray.get(cleanup_immns.remote(immunizations, is_train=True))

In [ ]:
for df in train_imm_cleaned:
    display(df.head(3))

,date,code
patient,,
ce9bd436-6b59-0452-86a4-61f3642736bc,2005-07-08 20:04:40,140
b8eb8d31-1031-fb5b-e207-b9815f80744c,2012-01-21 13:11:33,140
8b0484cd-3dbd-8b8d-1b72-a32f74a5a846,2012-05-06 08:57:56,140


,code,desc
0,140,Influenza seasonal injectable preservative ...
1,140,Influenza seasonal injectable preservative ...
2,140,Influenza seasonal injectable preservative ...


## Extract Labels (y)

The labels we intend to predict are conditions and must be in the `CONDITIONS` dict
- Adding them to the `patients` df
- And adding the patient's age when the particular condition was recorded

In [ ]:
for key in CONDITIONS.keys():
    print(key,"::",f'{CONDITIONS[key]}||START')

diabetes :: 44054006||START
stroke :: 230690007||START
alzheimers :: 26929004||START
coronary_heart :: 53741008||START
lung_cancer :: 254637007||START
breast_cancer :: 254837009||START
rheumatoid_arthritis :: 69896004||START
epilepsy :: 84757009||START


In [ ]:
#export
@ray.remote
def extract_ys(patients, conditions, cnd_dict):
    '''Extract labels from conditions df and add them to patients df with age.'''
    for key in cnd_dict.keys():
        patients = patients.merge(conditions[conditions.code==f'{cnd_dict[key]}||START'], how='left', left_index=True, right_index=True)
        patients[f'{key}'] = patients.code.notna()
        patients[f'{key}_age'] = ((patients.date - patients.birthdate)//np.timedelta64(1,'Y'))
        patients = patients.drop(columns=['date','code'])
    return patients

In [ ]:
tmp_pts = ray.get(extract_ys.remote(train_pts_cleaned[0], train_cnd_cleaned[0], cnd_dict=CONDITIONS))

In [ ]:
tmp_pts.count()

birthdate                   2123
diabetes                    2123
diabetes_age                 221
stroke                      2123
stroke_age                   444
alzheimers                  2123
alzheimers_age               283
coronary_heart              2123
coronary_heart_age           336
lung_cancer                 2123
lung_cancer_age               72
breast_cancer               2123
breast_cancer_age             82
rheumatoid_arthritis        2123
rheumatoid_arthritis_age       6
epilepsy                    2123
epilepsy_age                  47
dtype: int64

## Insert Age

Inserting patient's age in months and years into each record df
- this can be modified to records the patient's age in **days** or even **hours** that might be more relevant for datasets involving hospitalizations or ER admissions 

In [ ]:
#export
@ray.remote
def insert_age(df, pts_df):
    '''Insert age in years and months into each of the rec dfs'''
    
    df = df.merge(pts_df, left_index=True, right_index=True)
    df['age']        = (df['date'] - df['birthdate'])//np.timedelta64(1,'Y')
    df['age_months'] = (df['date'] - df['birthdate'])//np.timedelta64(1,'M')
    return df.drop(columns=['date','birthdate'])

### Clean all

In [ ]:
#export
@ray.remote(num_returns=2)
def clean_preprocess_dataset(path, is_train, conditions_dict, today=None):
    '''Cleans and preprocesses all dfs in a single split'''
    dfs = read_raw_ehrdata(path)
 
    pt_data   = cleanup_pts.remote(dfs[0],   is_train, today)
    obs_data  = cleanup_obs.remote(dfs[1],   is_train)
    alg_data  = cleanup_algs.remote(dfs[2],  is_train)
    crpl_data = cleanup_crpls.remote(dfs[3], is_train)
    med_data  = cleanup_meds.remote(dfs[4],  is_train)
    img_data  = cleanup_img.remote(dfs[5],   is_train)
    proc_data = cleanup_procs.remote(dfs[6], is_train)
    cnd_data  = cleanup_cnds.remote(dfs[7],  is_train)
    imm_data  = cleanup_immns.remote(dfs[8], is_train)   
    
    data_tables = [pt_data[0], pt_data[1], obs_data[0], alg_data[0], crpl_data[0], med_data[0], img_data[0], proc_data[0], cnd_data[0], imm_data[0]]
    
    patients, patient_demographics, conditions, rec_tables = data_tables[0], data_tables[1], data_tables[8], data_tables[2:]
    rec_dfs = [insert_age.remote(rec_df, patients) for rec_df in rec_tables]
    patients = extract_ys.remote(patients, conditions, conditions_dict)
    
    data_tables = [patients, patient_demographics]
    data_tables.extend(rec_dfs)
    
    if is_train:
        code_tables = [pt_data[2], obs_data[1], alg_data[1], crpl_data[1], med_data[1], img_data[1], proc_data[1], cnd_data[1], imm_data[1]]
    
    return (data_tables, code_tables) if is_train else (data_tables, None)

In [ ]:
data_tables, _ = ray.get(clean_preprocess_dataset.remote(f'{COHERENT_DATA_STORE}/raw_split/valid', is_train=False, conditions_dict=CONDITIONS))

patients, pt_demographics, observations, allergies, \
careplans, medications, imaging_studies, procedures, conditions, immunizations = ray.get(data_tables)

In [ ]:
conditions.count()

code          9913
age           9913
age_months    9913
dtype: int64

In [ ]:
data_tables, code_tables = ray.get(clean_preprocess_dataset.remote(f'{COHERENT_DATA_STORE}/raw_split/train', is_train=True, conditions_dict=CONDITIONS))

patients, pt_demographics, observations, allergies, \
careplans, medications, imaging_studies, procedures, conditions, immunizations = ray.get(data_tables)

pt_codes, obs_codes, alg_codes, crpl_codes, med_codes, img_codes, proc_codes, cnd_codes, imm_codes = ray.get(code_tables)

In [ ]:
conditions.count()

code          29583
age           29583
age_months    29583
dtype: int64

In [ ]:
obs_codes.count()

orig_code    845555
desc         845555
value        845555
units        845555
type         845555
dtype: int64

## Do-All Functions
The actual functions that will be called from other modules

In [ ]:
#export
@ray.remote
def persist_cleaned(path, split_name, cleaned_dfs, code_tables=None):
    '''Save cleaned EHR data to disk'''
    csv_names = FILENAMES.copy()
    csv_names.insert(1,'patient_demographics')
            
    cleaned_dir = Path(f'{path}/cleaned/{split_name}')
    cleaned_dir.mkdir(parents=True, exist_ok=True)
    
    cleaned_dfs = ray.get(cleaned_dfs)
    
    patients = cleaned_dfs[0]
    patients.reset_index(inplace=True)
    patients.to_csv(f'{cleaned_dir}/patients.csv', index_label='indx')

    for df, name in zip(cleaned_dfs[1:], csv_names[1:]):
        df.to_csv(f'{cleaned_dir}/{name}.csv')

    print(f'Saved cleaned "{split_name}" data to {cleaned_dir}')
        
    if split_name == 'train':
        codes_dir = Path(f'{cleaned_dir}/codes')
        codes_dir.mkdir(parents=True, exist_ok=True)
        
        code_tables = ray.get(code_tables)
        for code_df,name in zip(code_tables, FILENAMES):
            code_df.to_csv(f'{codes_dir}/code_{name}.csv', index_label='indx')
        print(f'Saved vocab code tables to {codes_dir}')
    return split_name

In [ ]:
#export
def clean_raw_ehrdata(path, valid_pct, test_pct, conditions_dict, today=None):
    '''Split, clean, preprocess raw EHR data & save cleaned data to disk'''
    
    # split
    split_ehr_dataset(path, valid_pct, test_pct)
    
    # clean + preprocess
    all_splits = []
    for split in ['train', 'valid', 'test']:
        split_path = f'{path}/raw_split/{split}'
        
        if split == 'train': data_tables, code_tables = clean_preprocess_dataset.remote(split_path, True,  conditions_dict, today)
        else               : data_tables, _           = clean_preprocess_dataset.remote(split_path, False, conditions_dict, today)
            
        all_splits.append(data_tables)
    
    # persist
    remaining = []
    remaining.append(persist_cleaned.remote(path, 'train', all_splits[0], code_tables))
    remaining.append(persist_cleaned.remote(path, 'valid', all_splits[1]))
    remaining.append(persist_cleaned.remote(path, 'test',  all_splits[2]))
    
    while len(remaining) > 0:
        ready, remaining = ray.wait(remaining)
        for r in ready:
            split_completed = ray.get(r)
            print(f'Completed - {split_completed}')
    return

In [ ]:
clean_raw_ehrdata(COHERENT_DATA_STORE, 0.2, 0.2, CONDITIONS, COHERENT_DATAGEN_DATE)

Splits:: train: 0.6, valid: 0.2, test: 0.2
Split patients into:: Train: 2123, Valid: 708, Test: 708 -- Total before split: 3539
Saved train data to /home/vinod/code/datasets/coherent/raw_split/train
Saved valid data to /home/vinod/code/datasets/coherent/raw_split/valid
Saved test data to /home/vinod/code/datasets/coherent/raw_split/test
Completed - valid
Completed - test
(pid=34379) Saved cleaned "test" data to /home/vinod/code/datasets/coherent/cleaned/test
(pid=34382) Saved cleaned "valid" data to /home/vinod/code/datasets/coherent/cleaned/valid
(pid=34380) Saved cleaned "train" data to /home/vinod/code/datasets/coherent/cleaned/train
(pid=34380) Saved vocab code tables to /home/vinod/code/datasets/coherent/cleaned/train/codes
Completed - train


In [ ]:
#export
def load_cleaned_ehrdata(path):
    '''Load cleaned, age-filtered EHR data'''
    
    csv_names = FILENAMES.copy()
    csv_names.insert(1,'patient_demographics')
    
    train_dfs = [pd.read_csv(f'{path}/cleaned/train/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
    valid_dfs = [pd.read_csv(f'{path}/cleaned/valid/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
    test_dfs  = [pd.read_csv(f'{path}/cleaned/test/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
                             
    return train_dfs, valid_dfs, test_dfs

In [ ]:
#export
def load_ehr_vocabcodes(path):
    '''Load codes for vocabs'''
    
    code_dfs = [pd.read_csv(f'{path}/cleaned/train/codes/code_{fname}.csv', low_memory=False, na_filter=False, index_col=0) for fname in FILENAMES]
                             
    return code_dfs

In [ ]:
train_dfs, valid_dfs, test_dfs = load_cleaned_ehrdata(COHERENT_DATA_STORE)
code_dfs = load_ehr_vocabcodes(COHERENT_DATA_STORE)

In [ ]:
# for df in train_dfs:
#     display(df.head())

In [ ]:
thispt = train_dfs[0].iloc[10]

In [ ]:
thispt

patient                     1f7a33f8-5cdd-61ca-0b3d-2af6fad8d08f
birthdate                                             1988-07-20
diabetes                                                   False
diabetes_age                                                 NaN
stroke                                                     False
stroke_age                                                   NaN
alzheimers                                                 False
alzheimers_age                                               NaN
coronary_heart                                             False
coronary_heart_age                                           NaN
lung_cancer                                                False
lung_cancer_age                                              NaN
breast_cancer                                              False
breast_cancer_age                                            NaN
rheumatoid_arthritis                                       False
rheumatoid_arthritis_age 

In [ ]:
# for df in code_dfs:
#     display(df.head())

Making sure condition counts match - after extracting `y` for each patient

In [ ]:
CONDITIONS

{'diabetes': '44054006',
 'stroke': '230690007',
 'alzheimers': '26929004',
 'coronary_heart': '53741008',
 'lung_cancer': '254637007',
 'breast_cancer': '254837009',
 'rheumatoid_arthritis': '69896004',
 'epilepsy': '84757009'}

`patients` dfs after cleaning, with `y` extracted

In [ ]:
pts_train, pts_valid, pts_test = train_dfs[0], valid_dfs[0], test_dfs[0]

`conditions` dfs

In [ ]:
cnd_train, cnd_valid, cnd_test = train_dfs[8], valid_dfs[8], test_dfs[8]

Tests to ensure counts match

In [ ]:
def test_extract_ys(pt_dfs, cnd_dfs, conditions_dict=CONDITIONS):
    """Test for extract_ys function."""
    for pts_df, cnds_df, split in zip(pt_dfs, cnd_dfs, ['train','valid','test']):
        print(f"Checking {split} dfs...")
        for this_cnd in conditions_dict.keys():
            code = f"{conditions_dict[this_cnd]}||START"
            cnds_df_counts = len(cnds_df[cnds_df['code'] == code])
            pts_df_counts = len(pts_df[pts_df[this_cnd] == 1])
            assert cnds_df_counts == pts_df_counts, f"Error in {split} for {this_cnd} -- {cnds_df_counts} != {pts_df_counts}"

        print(f"Tests passed for {split} - all condition counts match")
    return

In [ ]:
test_extract_ys([pts_train, pts_valid, pts_test],[cnd_train, cnd_valid, cnd_test])

Checking train dfs...
Tests passed for train - all condition counts match
Checking valid dfs...
Tests passed for valid - all condition counts match
Checking test dfs...
Tests passed for test - all condition counts match


In [ ]:
# export

def get_label_counts(pt_dfs, conditions_dict=CONDITIONS):
    """Get label counts in the given split of the dataset."""
    all_counts = []
    for pts_df, split in zip(pt_dfs, ['train','valid','test']):
        split_counts = {}
        for this_cnd in conditions_dict.keys():
            split_counts[this_cnd] = len(pts_df[pts_df[this_cnd] == 1])
        all_counts.append(split_counts)
    
    return all_counts

In [ ]:
get_label_counts([pts_train, pts_valid, pts_test])

[{'diabetes': 221,
  'stroke': 444,
  'alzheimers': 283,
  'coronary_heart': 336,
  'lung_cancer': 72,
  'breast_cancer': 82,
  'rheumatoid_arthritis': 6,
  'epilepsy': 47},
 {'diabetes': 80,
  'stroke': 158,
  'alzheimers': 76,
  'coronary_heart': 111,
  'lung_cancer': 27,
  'breast_cancer': 29,
  'rheumatoid_arthritis': 8,
  'epilepsy': 20},
 {'diabetes': 81,
  'stroke': 159,
  'alzheimers': 103,
  'coronary_heart': 118,
  'lung_cancer': 28,
  'breast_cancer': 32,
  'rheumatoid_arthritis': 6,
  'epilepsy': 17}]

In [ ]:
COHERENT_DATA_STORE

'/home/vinod/code/datasets/coherent'

In [ ]:
import json

In [ ]:
with open(f'{COHERENT_DATA_STORE}/output/fhir/Abe604_Frami345_b8dd1798-beef-094d-1be4-f90ee0e6b7d5.json') as f:
    data = json.load(f)

In [ ]:
data['entry']

[{'fullUrl': 'urn:uuid:b8dd1798-beef-094d-1be4-f90ee0e6b7d5',
  'resource': {'resourceType': 'Patient',
   'id': 'b8dd1798-beef-094d-1be4-f90ee0e6b7d5',
   'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']},
   'text': {'status': 'generated',
    'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: v2.6.1-174-g66c40fa7\n .   Person seed: 771024127226607103  Population seed: 1626964256551</div>'},
   'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',
     'extension': [{'url': 'ombCategory',
       'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
        'code': '2106-3',
        'display': 'White'}},
      {'url': 'text', 'valueString': 'White'}]},
    {'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity',
     'extension': [{'url': 'ombCategory',
       'valueCoding': {'system': 'urn:oi